In [1]:
import pandas as pd
import numpy as np
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig
from azureml.core.compute import AmlCompute,ComputeTarget, ComputeInstance
from azureml.exceptions import ComputeTargetException
from azureml.core.datastore import Datastore
from azureml.widgets import RunDetails
from azureml.core.environment import CondaDependencies

In [3]:
ws = Workspace.from_config(path='./config/config.json')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [6]:
env=Environment.from_pip_requirements("conv_sum",  './config/requirements.txt')
env.register(ws)

{
    "assetId": "azureml://locations/eastus/workspaces/0eba8fae-31fb-491d-b2c5-a2b9ff37696b/environments/conv_sum/versions/3",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20221101.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "conv_sum",
    "python": {
  

In [11]:
from utils.ComputeManagement import create_cluster, create_instance, delete_compute

In [27]:
compute=create_cluster(
    workspaceRef=ws,
    name="testcompute2", # Do not put _
    vmSize="Standard_DS3_v2",
    minNodes=1,
    maxNodes=4,
    idleTime=180
)

Workspace.create(name='AzureMLWorkspace', subscription_id='5d2e45e0-cd7b-4338-b279-455fa4a4c42d', resource_group='RG') testcompute2 Standard_DS3_v2 1 4 180
Compute target already exists. Using it.


In [34]:
compute.list_nodes()

[{'nodeId': 'tvmps_83753f9d2ab68174a678eacb394f6ace8c5d153815538f2133216f550aa02071_d',
  'port': 50001,
  'publicIpAddress': '20.246.146.164',
  'privateIpAddress': '10.0.0.5',
  'nodeState': 'idle'}]

In [51]:
training_script_config = ScriptRunConfig(
    source_directory = 'src',
    script = 'TrainingScript.py',
    arguments=["--filePath","../data/ril.csv"],
    environment = env,
    compute_target = compute
)
experiment = Experiment(
    workspace = ws,
    name="maiden_experiment"
)
run = experiment.submit(config=training_script_config)

RunDetails(run).show()
run.wait_for_completion(show_output=True)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

RunId: maiden_experiment_1674285148_183dda74
Web View: https://ml.azure.com/runs/maiden_experiment_1674285148_183dda74?wsid=/subscriptions/5d2e45e0-cd7b-4338-b279-455fa4a4c42d/resourcegroups/RG/workspaces/AzureMLWorkspace&tid=d390d3b6-5b59-47f2-a790-59a4e06413d3

Streaming user_logs/std_log.txt

OS Path /mnt/azureml/cr/j/31a59a99903d4f0abaf8d9070d5757f1/exe/wd
Cleaning up all outstanding Run operations, waiting 300.0 seconds
1 items cleaning up...
Cleanup took 0.07273554801940918 seconds
Traceback (most recent call last):
  File "TrainingScript.py", line 97, in <module>
    train(args)
  File "TrainingScript.py", line 40, in train
    data = pd.read_csv(args.filePath).values.tolist()
  File "/azureml-envs/azureml_80e311665d5b8f75e420bb60aaebc26a/lib/python3.8/site-packages/pandas/util/_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "/azureml-envs/azureml_80e311665d5b8f75e420bb60aaebc26a/lib/python3.8/site-packages/pandas/util/_decorators.py", line 331, in 

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Execution failed. User process '/azureml-envs/azureml_80e311665d5b8f75e420bb60aaebc26a/bin/python' exited with status code 1. Please check log file 'user_logs/std_log.txt' for error details. Error:   File \"TrainingScript.py\", line 97, in <module>\n    train(args)\n  File \"TrainingScript.py\", line 40, in train\n    data = pd.read_csv(args.filePath).values.tolist()\n  File \"/azureml-envs/azureml_80e311665d5b8f75e420bb60aaebc26a/lib/python3.8/site-packages/pandas/util/_decorators.py\", line 211, in wrapper\n    return func(*args, **kwargs)\n  File \"/azureml-envs/azureml_80e311665d5b8f75e420bb60aaebc26a/lib/python3.8/site-packages/pandas/util/_decorators.py\", line 331, in wrapper\n    return func(*args, **kwargs)\n  File \"/azureml-envs/azureml_80e311665d5b8f75e420bb60aaebc26a/lib/python3.8/site-packages/pandas/io/parsers/readers.py\", line 9

In [50]:
import os
os.getcwd()

'/Users/anupam/Documents/Codebase/MLOps_stock_prediction'

In [40]:
delete_compute(workspaceRef=ws, computeToDelete="testcompute2")

Compute target does not exist.
